In [27]:
import numpy as np
from scipy.stats import poisson

In [28]:
M = 10000  # Number of independent samples
threshold = 20  

###  (i)

In [29]:
def crude_monte_carlo(M):
    U_samples = np.random.uniform(0, 1, M)
    mu_values = 15 / (0.5 + U_samples)
    N_samples = np.random.poisson(mu_values)
    crude_probs = N_samples >= threshold
    return np.mean(crude_probs), np.var(crude_probs) / M


In [30]:
p_crude, var_crude = crude_monte_carlo(M)
print("Crude Monte Carlo:")
print(f"Estimate: {p_crude:.6f}")

Crude Monte Carlo:
Estimate: 0.289700


### (ii)

In [31]:
def conditional_probability(u):
    mu = 15 / (0.5 + u)
    return 1 - poisson.cdf(19, mu)  # P(N >= 20 | U)

In [32]:
def conditional_with_control_variate(M):
    U_samples = np.random.uniform(0, 1, M)
    # Compute conditional probabilities and control variates
    conditional_probs = np.array([conditional_probability(u) for u in U_samples])
    control_variates = 15 / (0.5 + U_samples)  # Mean of the Poisson as the control variate

    # Control variate adjustment
    control_mean = np.mean(control_variates)  # Empirical mean of the control variate
    control_var = np.var(control_variates)  # Variance of the control variate
    covariance = np.cov(conditional_probs, control_variates)[0, 1]  # Covariance
    beta = covariance / control_var  # Optimal beta

    # Adjusted estimator
    adjusted_probs = conditional_probs - beta * (control_variates - control_mean)
    return np.mean(adjusted_probs), np.var(adjusted_probs) / M

In [33]:
p_conditional, var_conditional = conditional_with_control_variate(M)
print("\nConditional Expectation:")
print(f"Estimate: {p_conditional:.6f}")


Conditional Expectation:
Estimate: 0.288039


### (iii)

In [34]:
def antithetic_variables(M):
    U_samples = np.random.uniform(0, 1, M // 2)  # Half the samples (pairs are combined)
    U_antithetic = 1 - U_samples
    probs_original = np.array([conditional_probability(u) for u in U_samples])
    probs_antithetic = np.array([conditional_probability(u) for u in U_antithetic])
    combined_probs = (probs_original + probs_antithetic) / 2
    return np.mean(combined_probs), np.var(combined_probs) / (M // 2)


In [35]:
p_antithetic, var_antithetic = antithetic_variables(M)
print("\nAntithetic Variables:")
print(f"Estimate: {p_antithetic:.6f}")


Antithetic Variables:
Estimate: 0.286782


In [36]:
print("Crude Monte Carlo:")
print(f"Estimate: {p_crude:.6f}, Variance: {var_crude:.6e}")

print("\nConditional Expectation:")
print(f"Estimate: {p_conditional:.6f}, Variance: {var_conditional:.6e}")

print("\nAntithetic Variables:")
print(f"Estimate: {p_antithetic:.6f}, Variance: {var_antithetic:.6e}")



Crude Monte Carlo:
Estimate: 0.289700, Variance: 2.057739e-05

Conditional Expectation:
Estimate: 0.288039, Variance: 3.241113e-07

Antithetic Variables:
Estimate: 0.286782, Variance: 3.233519e-06
